# Baseline Classifier

In [1]:
! gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?code_challenge=FvKaxJodnRh9SR9h2g3ENbSDX7XGr_mT7c-sbz5HwY4&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [galli.giuly@gmail.com].
Your current project is [reddit-master].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [2]:
!gcloud config set project reddit-master

Updated property [core/project].


In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import plotly.graph_objs as go
import chart_studio.plotly as py
import cufflinks
import plotly.figure_factory as ff
import logging

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score


Using TensorFlow backend.


In [4]:
!gsutil ls gs://reddit_final_results/

gs://reddit_final_results/comments_posts_2018.csv
gs://reddit_final_results/comments_posts_2018.zip
gs://reddit_final_results/comments_posts_2018_V2.csv
gs://reddit_final_results/comments_posts_2018_V2.zip
gs://reddit_final_results/comments_posts_Fitness.pkl
gs://reddit_final_results/comments_posts_IAmA.pkl
gs://reddit_final_results/comments_posts_atheism.pkl
gs://reddit_final_results/comments_posts_aww.pkl
gs://reddit_final_results/comments_posts_europe.pkl
gs://reddit_final_results/comments_posts_funny.pkl
gs://reddit_final_results/comments_posts_gaming.pkl
gs://reddit_final_results/comments_posts_movies.pkl
gs://reddit_final_results/comments_posts_nba.pkl
gs://reddit_final_results/comments_posts_politics.pkl
gs://reddit_final_results/comments_posts_science.pkl
gs://reddit_final_results/comments_posts_technology.pkl
gs://reddit_final_results/comments_posts_todayilearned.pkl
gs://reddit_final_results/comments_posts_tokenized.csv
gs://reddit_final_results/comments_posts_tokenized.zip
g

In [5]:
subreddits = ['aww', 'nba', 'movies', 'todayilearned', 'IAmA', 'Fitness', 'worldnews', 'technology', 'europe', 'politics', 'atheism','science', 'funny', 'gaming']

In [6]:
%%time

model_baseline = pd.DataFrame(columns = ["subreddit", "body", "subreddit_id"])

for subreddit in subreddits:
    !gsutil cp gs://reddit_final_results/comments_posts_{subreddit}.pkl .
    print("comments_posts_" + subreddit + ".pkl downloaded")

    df = pd.read_pickle("comments_posts_" + subreddit + ".pkl")

    model_baseline = pd.concat([model_baseline, df], ignore_index=True)

Copying gs://reddit_final_results/comments_posts_aww.pkl...
/ [1 files][ 63.8 MiB/ 63.8 MiB]                                                
Operation completed over 1 objects/63.8 MiB.                                     
comments_posts_aww.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




Copying gs://reddit_final_results/comments_posts_nba.pkl...
- [1 files][ 88.3 MiB/ 88.3 MiB]    6.9 MiB/s                                   
Operation completed over 1 objects/88.3 MiB.                                     
comments_posts_nba.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




Copying gs://reddit_final_results/comments_posts_movies.pkl...
\ [1 files][103.2 MiB/103.2 MiB]    5.7 MiB/s                                   
Operation completed over 1 objects/103.2 MiB.                                    
comments_posts_movies.pkl downloaded


...

Copying gs://reddit_final_results/comments_posts_funny.pkl...
\ [1 files][ 69.3 MiB/ 69.3 MiB]    4.5 MiB/s                                   
Operation completed over 1 objects/69.3 MiB.                                     
comments_posts_funny.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




Copying gs://reddit_final_results/comments_posts_gaming.pkl...
/ [1 files][ 79.4 MiB/ 79.4 MiB]                                                
Operation completed over 1 objects/79.4 MiB.                                     
comments_posts_gaming.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




CPU times: user 2min 2s, sys: 2min 5s, total: 4min 8s
Wall time: 9min 3s


In [7]:
X = model_baseline['body']
y = model_baseline['subreddit_id']

# Definint a fucntion that slit the dataset into three subsets: train, val and test
def train_dev_test_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    return dict(X_train=X_train,
                X_test=X_test,
                y_train=y_train.astype('int'),
                y_test=y_test.astype('int'))

model_baseline_split = train_dev_test_split(X, y)

In [8]:
dummy = DummyClassifier()

dummy.fit(model_baseline_split["X_train"], model_baseline_split["y_train"])

y_pred = dummy.predict(model_baseline_split["X_test"])

DummyClassifier(constant=None, random_state=None, strategy='stratified')

In [9]:
print(accuracy_score(y_pred, model_baseline_split["y_test"]))

0.07130635183638179


#### Prueba de Multi Layer Perceptron con Keras